# Config generator for Home Assistant

In [13]:
import yaml

## Stale automations

Detect stale sensor readings and shut down associated thermostats

Example:

```yaml
- id: '1604254268934'
  alias: 'Stale sensor: Pip''s Room'
  description: ''
  trigger:
    - platform: time_pattern
      minutes: 7
  condition:
  - condition: and
    conditions:
    - condition: template
      value_template: '{{ (as_timestamp(now()) - as_timestamp(states.sensor.pip_temperature.last_changed))/(60*60) > 12 }}'
    - condition: state
      entity_id: automation.turn_pip_s_heating_on
      state: 'on'
  action:
  - data: {}
    entity_id: automation.turn_pip_s_heating_on
    service: automation.turn_off
  - data:
      message: Disabling Pip Heating
      title: 'Stale sensor: pip'
    service: notify.mobile_app_pixel_2
  mode: single
```

### Simple sensors 1:1 control automation

These have a single sensor and a single automation that turns on the heating.

- When stale, turn off the automation.
- When fresh, turn on the automation.

In [134]:
# escape any ' with ''
settings = [
    {"id": "pip_room_stale", 
     "alias": "Pip''s room", 
     "sensor": "pip_temperature", 
     "timeout": 4, 
     "automation": "turn_pip_s_heating_on"},
    {"id": "kitchen_stale", 
     "alias": "Kitchen", 
     "sensor": "kitchen_temperature", 
     "timeout": 4, 
     "automation": "kitchen_heating_on"}    
]

In [131]:
config=""
for item in settings:
    config += f"""
- id: '{item["id"]}'
  alias: 'Stale sensor: {item["alias"]}'
  description: ''
  trigger:
    - platform: time_pattern
      minutes: 7
  condition:
  - condition: and
    conditions:
    - condition: template
      value_template: '{{ (as_timestamp(now()) - as_timestamp(states.sensor.{item["sensor"]}.last_changed))/(60*60) > {item["timeout"]} }}'
    - condition: state
      entity_id: automation.{item["automation"]}
      state: 'on'
  action:
  - data: {{}}
    entity_id: automation.{item["automation"]}
    service: automation.turn_off
  - data:
      message: Disabling {item["alias"]}
      title: 'Stale sensor: {item["sensor"]}'
    service: notify.mobile_app_pixel_2
  mode: single
- id: '{item["id"]}_enable'
  alias: 'Fresh sensor: {item["alias"]}'
  description: ''
  trigger:
    - platform: time_pattern
      minutes: 8
  condition:
  - condition: and
    conditions:
    - condition: template
      value_template: '{{ (as_timestamp(now()) - as_timestamp(states.sensor.{item["sensor"]}.last_changed))/(60*60) < {item["timeout"]} }}'
    - condition: state
      entity_id: automation.{item["automation"]}
      state: 'off'
  action:
  - data: {{}}
    entity_id: automation.{item["automation"]}
    service: automation.turn_on
  - data:
      message: Enabling {item["alias"]}
      title: 'Fresh sensor: {item["sensor"]}'
    service: notify.mobile_app_pixel_2
  mode: single
    """

In [132]:
try:
    yaml.safe_load(config)
    print("YAML parsed successfully")
except:
    print("YAML failed to parse")

YAML parsed successfully


In [133]:
print(config)


- id: 'pip_room_stale'
  alias: 'Stale sensor: Pip''s room'
  description: ''
  trigger:
    - platform: time_pattern
      minutes: 7
  condition:
  - condition: and
    conditions:
    - condition: template
      value_template: '{ (as_timestamp(now()) - as_timestamp(states.sensor.pip_temperature.last_changed))/(60*60) > 4 }'
    - condition: state
      entity_id: automation.turn_pip_s_heating_on
      state: 'on'
  action:
  - data: {}
    entity_id: automation.turn_pip_s_heating_on
    service: automation.turn_off
  - data:
      message: Disabling Pip''s room
      title: 'Stale sensor: pip_temperature'
    service: notify.mobile_app_pixel_2
  mode: single
- id: 'pip_room_stale_enable'
  alias: 'Fresh sensor: Pip''s room'
  description: ''
  trigger:
    - platform: time_pattern
      minutes: 8
  condition:
  - condition: and
    conditions:
    - condition: template
      value_template: '{ (as_timestamp(now()) - as_timestamp(states.sensor.pip_temperature.last_changed))/(60*60